## Set WD

In [1]:
import os
os.chdir("/home/sgf2/DBMI_server/adversarial_attacks/cxr/")

In [1]:
import scipy.stats as st
import tensorflow.keras 
from sklearn import metrics
from sklearn.metrics import auc

def mean_ci(x):
    mn = np.mean(x)
    ci = st.t.interval(0.95, len(x)-1, loc=np.mean(x), scale=st.sem(x))
    return (mn, ci[0], ci[1])

def printResults(model_preds, y_test):
    acc = np.mean(np.round(model_preds)[:,0] == y_test[:,0])
    print('Test accuracy: %0.4f' % acc)

    fpr, tpr, thresholds = metrics.roc_curve(y_test[:,1], model_preds[:,1])
    auc_score = auc(fpr,tpr)
    print('AUC: %0.4f' % auc_score)

    conf = mean_ci(np.max(model_preds, axis = 1))
    print('Avg. Confidence: ' + '{0:.6f} '.format(conf[0]) + \
          '({0:.6f}'.format(conf[1]) + ' - {0:.6f})'.format(conf[2]))

## Set up

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
from craft_attack_patch import *

# Adversarial Patch Attack

## Train Patches

In [3]:
# Hyperparameters
epochs = 7
learning_rate = 5.0

# Load the models
resnet1 = ModelContainer('resnet1')
resnet2 = ModelContainer('resnet2')

# White Box
model = resnet1









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://

In [4]:
# Target 0
train(model, target_label=0, epochs=epochs, learning_rate=learning_rate)
file_name = './patches/resnet1_patch_target0_epoch' + str(epochs) + '_wb.npy'
np.save(file_name, model.patch())

# Target 1
train(model, target_label=1, epochs=epochs, learning_rate=learning_rate)
file_name = './patches/resnet1_patch_target1_epoch' + str(epochs) + '_wb.npy'
np.save(file_name, model.patch())

# Black Box
model = resnet2

# Target 0
train(model, target_label=0, epochs=epochs, learning_rate=learning_rate)
file_name = './patches/resnet1_patch_target0_epoch' + str(epochs) + '_bb.npy'
np.save(file_name, model.patch())

# Target 1
train(model, target_label=1, epochs=epochs, learning_rate=learning_rate)
file_name = './patches/resnet1_patch_target1_epoch' + str(epochs) + '_bb.npy'
np.save(file_name, model.patch())

(minibatch 0) loss: 0.07890727
(minibatch 1) loss: 0.047167458
Loss after epoch 0: 0.06303736567497253
(minibatch 0) loss: 0.06889289
(minibatch 1) loss: 0.031685874
Loss after epoch 1: 0.05028938129544258
(minibatch 0) loss: 0.024831343
(minibatch 1) loss: 0.024010384
Loss after epoch 2: 0.024420863948762417
(minibatch 0) loss: 0.034733854
(minibatch 1) loss: 0.011770058
Loss after epoch 3: 0.023251956328749657
(minibatch 0) loss: 0.017316163
(minibatch 1) loss: 0.017208152
Loss after epoch 4: 0.017262157052755356
(minibatch 0) loss: 0.010074705
(minibatch 1) loss: 0.035962198
Loss after epoch 5: 0.023018451407551765
(minibatch 0) loss: 0.021577694
(minibatch 1) loss: 0.032837998
Loss after epoch 6: 0.02720784582197666
(minibatch 0) loss: 2.6175125
(minibatch 1) loss: 3.707717
Loss after epoch 0: 3.1626147031784058
(minibatch 0) loss: 3.4173436
(minibatch 1) loss: 2.0859015
Loss after epoch 1: 2.7516225576400757
(minibatch 0) loss: 2.6628585
(minibatch 1) loss: 2.3804598
Loss after ep

## Display Results

In [5]:
len(image_loader.X_test)

16

In [6]:
################################# THIS IS THE ONLY CELL THAT WAS ADDED #############
# Pick the images to show (attack). Here I randomly chose 2 from each class
show(image_loader.X_test[3]) # label 0
show(image_loader.X_test[1]) # label 0 
show(image_loader.X_test[7]) # label 1
show(image_loader.X_test[15]) # label 1

In [7]:
################################# THIS IS THE ONLY CELL THAT WAS MODIFIED #############
epochs = 7
# Load the models
resnet1 = ModelContainer('resnet1')
resnet2 = ModelContainer('resnet2')

# Loading the patch file 
resnet1_patch_target1_wb = np.load('./patches/resnet1_patch_target1_epoch' + str(epochs) + '_wb.npy')
resnet1_patch_target0_wb = np.load('./patches/resnet1_patch_target0_epoch' + str(epochs) + '_wb.npy')

Finished loading resnet1, took 11s
Finished loading resnet2, took 11s


In [8]:
# Indices of images to show (use n_show=0 when using show_indices feature)
show_indices0 = [9]#[7868] # attack with target label 0
show_indices1 = [5]#[384]

# Apply
scale = 0.4
probs_patched_images, probs_original_images,indices, true_labels, winp = attack_combined(resnet1,patch_for_0=resnet1_patch_target0_wb,
                                                                         patch_for_1=resnet1_patch_target1_wb,
                                                                         n_show=0, scale=scale,
                                                                         show_indices0=show_indices0,
                                                                         show_indices1=show_indices1,
                                                                         predict_original=False
                                                                                        )


# Print
print("White Box:")
printResults(probs_patched_images, keras.utils.to_categorical(true_labels))

9   [ 8  9 10 11 12 13 14 15]   [9]
5   [0 1 2 3 4 5 6 7]   [5]
White Box:
Test accuracy: 0.0000
AUC: 0.0000
Avg. Confidence: 0.951133 (0.916495 - 0.985771)


In [9]:
# Loading the patch file 
resnet1_patch_target1_bb = np.load('./patches/resnet1_patch_target1_epoch7_bb.npy')
resnet1_patch_target0_bb = np.load('./patches/resnet1_patch_target0_epoch7_bb.npy')

# Apply
probs_patched_images, probs_original_images, indices, true_labels, winp = attack_combined(resnet1,patch_for_0=resnet1_patch_target0_bb,
                                                                         patch_for_1=resnet1_patch_target1_bb,
                                                                         n_show=0, scale=scale,
                                                                         show_indices0=show_indices0,
                                                                         show_indices1=show_indices1,
                                                                         predict_original=False
                                                                                        )
# Print
print("\nBlack Box:")
printResults(probs_patched_images, keras.utils.to_categorical(true_labels))

9   [ 8  9 10 11 12 13 14 15]   [9]
5   [0 1 2 3 4 5 6 7]   [5]

Black Box:
Test accuracy: 0.0625
AUC: 0.0000
Avg. Confidence: 0.870792 (0.786580 - 0.955004)


# Photoshop Patch Attacks

In [10]:
# Combined attack with mole-patches
file_name0 = './patches/natural_patch_min_p.npy'
file_name1 = './patches/natural_patch_max_p.npy'
mole_patch0 = np.load(file_name0)
mole_patch1 = np.load(file_name1)
scale = 0.4
probs_patched_images, probs_original_images, indices, true_labels, winp = attack_combined(resnet1,patch_for_0=mole_patch0,
                                                                         patch_for_1=mole_patch1,
                                                                         n_show=0, scale=scale,
                                                                         show_indices0=show_indices0,
                                                                         show_indices1=show_indices1,
                                                                         predict_original=False
                                                                                        )
printResults(probs_patched_images, keras.utils.to_categorical(true_labels))

FileNotFoundError: [Errno 2] No such file or directory: './patches/natural_patch_min_p.npy'

Test accuracy: 0.0137
AUC: 0.0027
Avg. Confidence: 0.963693 (0.956826 - 0.970560)
